### CryoParticleSegment

In [ ]:
#%%bash
#git clone git+https://cyanazuki@github.com/cyanazuki/CryoParticleSegment.git
#cd CryoParticleSegment
#python setup.py install

## ⭐ Setup
You must run all codes under this category.

### ✅ Directory Settings

In [ ]:
# @title  { display-mode: "form" }

IMAGE_DIR = "/content/drive/MyDrive/research_xs/processed_micrographs_np" # @param {type:"string"}
LABEL_DIR = "/content/drive/MyDrive/research_xs/ground_truth_mask" # @param {type:"string"}
RESULT_DIR = "/content/drive/MyDrive/research_xs/final/deeplabv3_resnet50/model_only" # @param {type:"string"}

In [ ]:
# @title  { display-mode: "form" }
# @markdown Detect whether using folder in Google Drive as **`RESULT DIR`**📁.

if "content" in IMAGE_DIR.split("/")[:3] or "content" in LABEL_DIR.split("/")[:3]:
  try:
    from google.colab import drive
    drive.mount('/content/drive')
    !rm -r /content/sample_data
    if "content" in IMAGE_DIR.split("/")[:3]:
      !cp -r {IMAGE_DIR} /content/image_dir
      IMAGE_DIR = "/content/image_dir"
    if "content" in LABEL_DIR.split("/")[:3]:
      !cp -r {LABEL_DIR} /content/label_dir
      LABEL_DIR = "/content/label_dir"
  except:
    pass

Mounted at /content/drive


In [ ]:
# @title  { display-mode: "form" }
# @markdown Source code directory.
SRC_DIR = "/content/drive/MyDrive/research/src" # @param {type:"string"}

if True:
  !cp -r {SRC_DIR}/EM_project/*.py /content/
else:
  !cp {SRC_DIR}/EM_project/convcrf.py /content/convcrf.py
  !cp {SRC_DIR}/EM_project/dataset.py /content/dataset.py
  !cp {SRC_DIR}/EM_project/lr_scheduler.py /content/lr_scheduler.py
  !cp {SRC_DIR}/EM_project/metrics.py /content/metrics.py
  !cp {SRC_DIR}/EM_project/model.py /content/model.py
  !cp {SRC_DIR}/EM_project/trainer.py /content/trainer.py
  !cp {SRC_DIR}/EM_project/utils.py /content/utils.py

### ✅ Packages Handling

In [ ]:
# @title  { display-mode: "form" }
# @markdown Useful packages.

import os
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
# @title  { display-mode: "form" }
# @markdown User-defined packages.

from dataset import MicrographDataset, MicrographDatasetEvery
from model import create_model
from trainer import CryoEMTrainerWithScheduler, tqdm_plugin_for_Trainer, CryoEMEvaluator

## ⭐ Main

### ✅ Setting

In [ ]:
# @markdown Parameters.

NUM_CLASSES = 2
EPOCHS = 100
BATCH = 2
CROP_SIZE = (1024, 1024)
LR = 1e-3
RLR_PATIENCE = 3
ES_PATIENCE = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# @markdown Set seed.

random_state = 42
torch.manual_seed(random_state)
torch.cuda.manual_seed_all(random_state)

### ✅ Dataset

In [ ]:
train_dir = os.path.join(IMAGE_DIR, 'train')
train_filenames = np.loadtxt(f"{IMAGE_DIR}/train_filenames.txt", dtype=str)
train_dataset = MicrographDataset(image_dir=train_dir, label_dir=LABEL_DIR, filenames=train_filenames, crop_size=CROP_SIZE)

In [ ]:
val_dir = os.path.join(IMAGE_DIR, 'val')
val_filenames = np.loadtxt(f"{IMAGE_DIR}/val_filenames.txt", dtype=str)
val_dataset = MicrographDatasetEvery(image_dir=val_dir, label_dir=LABEL_DIR, filenames=val_filenames, crop_size=CROP_SIZE)
val_loader = DataLoader(val_dataset, batch_size=None, shuffle=False, pin_memory=True)

In [ ]:
test_dir = os.path.join(IMAGE_DIR, 'test')
test_filenames = np.loadtxt(f"{IMAGE_DIR}/test_filenames.txt", dtype=str)
test_dataset = MicrographDatasetEvery(image_dir=test_dir, label_dir=LABEL_DIR, filenames=test_filenames, crop_size=CROP_SIZE)
test_loader = DataLoader(test_dataset, batch_size=None, shuffle=False, pin_memory=True)

### ✅ Model

In [ ]:
backbone = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet50', pretrained=True)

backbone.backbone.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
backbone.classifier[4] = torch.nn.Conv2d(256, NUM_CLASSES, kernel_size=1, stride=1)
backbone.aux_classifier[4] = torch.nn.Conv2d(256, NUM_CLASSES, kernel_size=1, stride=1)

model = create_model(backbone)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet50_coco-cd0a2569.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet50_coco-cd0a2569.pth
100%|██████████| 161M/161M [00:03<00:00, 54.3MB/s]


In [ ]:
criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = ReduceLROnPlateau(optimizer, patience=RLR_PATIENCE, verbose=True)

## ⭐ Training

In [ ]:
Trainer = tqdm_plugin_for_Trainer(CryoEMTrainerWithScheduler)
trainer = Trainer(model, train_dataset, criterion, optimizer, device,
              num_classes = NUM_CLASSES,
              lr_scheduler=scheduler, patience=ES_PATIENCE)

trainer.train(EPOCHS, val_loader=val_loader, batch_size = BATCH,
              ckpt_dir = RESULT_DIR, random_state = random_state)

Epoch   1/100:


Training:   0%|          | 0/29 [00:00<?, ?it/s]

Training score:
  loss	: 0.5194


Validation:   0%|          | 0/9 [00:00<?, ?it/s]

Validation score:
  loss : 0.6832
global correct: 63.59
average row correct: ['76.98', '22.65']
Recall: ['75.26', '24.34']
IoU: ['61.44', '13.29']
F1 Score: ['76.11', '23.46']
mean IoU: 37.36
Saving model at /content/drive/MyDrive/research_xs/final/deeplabv3_resnet50/model_only/checkpoint1.pt
Loss improve to 0.6831786301400926.
Epoch   2/100:


...

Validation score:
  loss : 0.4440
global correct: 90.67
average row correct: ['96.72', '72.16']
Recall: ['91.40', '87.79']
IoU: ['88.65', '65.58']
F1 Score: ['93.98', '79.21']
mean IoU: 77.11
No improvement for 19 epoch.
Epoch  46/100:


Training:   0%|          | 0/29 [00:00<?, ?it/s]

Training score:
  loss	: 0.4302


Validation:   0%|          | 0/9 [00:00<?, ?it/s]

Validation score:
  loss : 0.4435
global correct: 90.71
average row correct: ['96.60', '72.70']
Recall: ['91.54', '87.50']
IoU: ['88.68', '65.86']
F1 Score: ['94.00', '79.42']
mean IoU: 77.27
Early stopping


In [ ]:
import pickle

trainer_result = {
    '_results': trainer._results,
    'best_epoch': trainer.best_epoch,
    'best_loss': trainer.best_loss,
    'loss': trainer.loss,
}

with open(f'{RESULT_DIR}/trainer_result.pickle', 'wb') as f:
    pickle.dump(trainer_result, f)

## ⭐ Evaluate

### ⏭ Test score for all saved checkpoint.

In [ ]:
checkpoint_paths = [path for path in os.listdir(RESULT_DIR) if '.pt' in path]
for checkpoint_path in checkpoint_paths:
  state_dict_path = f"{RESULT_DIR}/{checkpoint_path}"
  state_dict = torch.load(state_dict_path, map_location=torch.device(device))
  model.load_state_dict(state_dict, strict=False)
  model.eval()
  print("\nLoad model at: ", state_dict_path)
  trainer = CryoEMTrainerWithScheduler(model, train_dataset, criterion, optimizer, device,
                num_classes=NUM_CLASSES,
                lr_scheduler=scheduler, patience=ES_PATIENCE)
  result = trainer.evaluate(test_loader)


Load model at:  /content/drive/MyDrive/research_xs/final/deeplabv3_resnet101/checkpoint1.pt
  loss : 0.6303
  Average Precision : 0.0329
global correct: 75.38
average row correct: ['92.1', '17.6']
IoU: ['74.4', '13.8']
mean IoU: 44.07

Load model at:  /content/drive/MyDrive/research_xs/final/deeplabv3_resnet101/checkpoint3.pt
  loss : 0.5846
  Average Precision : 0.1017
global correct: 79.96
average row correct: ['87.3', '54.4']
IoU: ['77.2', '37.8']
mean IoU: 57.47

Load model at:  /content/drive/MyDrive/research_xs/final/deeplabv3_resnet101/checkpoint4.pt
  loss : 0.5604
  Average Precision : 0.1009
global correct: 81.71
average row correct: ['90.1', '52.5']
IoU: ['79.3', '39.1']
mean IoU: 59.18

Load model at:  /content/drive/MyDrive/research_xs/final/deeplabv3_resnet101/checkpoint5.pt
  loss : 0.5485
  Average Precision : 0.1058
global correct: 82.64
average row correct: ['90.7', '54.6']
IoU: ['80.2', '41.3']
mean IoU: 60.75

Load model at:  /content/drive/MyDrive/research_xs/fina

### ⏭ Load previous

In [ ]:
# @markdown Set show_checkpoint to True to check every checkpoint.
# @markdown If checkpoint_filename is "" (empty), the last checkpoint will be used.
show_checkpoint = False # @param {type:"boolean"}
checkpoint_filename = "" # @param {type:"string"}
if True:
  checkpoint_paths = [path for path in os.listdir(RESULT_DIR) if '.pt' in path]
  if show_checkpoint:
    print(checkpoint_paths)
  if str(checkpoint_filename) == "":
    checkpoint_filename = checkpoint_paths[-1]
  state_dict_path = f"{RESULT_DIR}/{checkpoint_filename}"
  state_dict = torch.load(state_dict_path, map_location=torch.device(device))
  model.load_state_dict(state_dict, strict=False)
  model.eval()
  print("Load model at: ", state_dict_path)
  try:
    # Run if changing trainer.
    import pickle
    # !rm __pycache__/*
    # !cp /content/drive/MyDrive/research/src/EM_project/trainer.py /content/trainer.py
    # from trainer import CryoEMTrainerWithScheduler
    Trainer = tqdm_plugin_for_Trainer(CryoEMTrainerWithScheduler)
    trainer = Trainer(model, train_dataset, val_dataset, criterion, optimizer, device,
                  num_classes = NUM_CLASSES,
                  lr_scheduler=scheduler, patience=ES_PATIENCE)
    with open(f'{RESULT_DIR}/trainer_result.pickle', 'rb') as f:
      trainer_result = pickle.load(f)
    for key in trainer_result:
      setattr(trainer, key, trainer_result[key])
    # for var in dir(trainer):
    #   if '__' not in var:
    #     print(var)
    #     print(f'var:\n  ', getattr(trainer, var))
  except:
    pass

Load model at:  /content/drive/MyDrive/research_xs/final/deeplabv3_resnet50/model_only/checkpoint26.pt


### ✅ Testing

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
evaluator = CryoEMEvaluator(model=model, device=device, metrics=['iou'], num_classes=NUM_CLASSES)
print("FCN validation result:")
result = evaluator.evaluate(loader=val_loader)
print("FCN test result:")
result = evaluator.evaluate(loader=test_loader)

FCN validation result:
global correct: 90.85
average row correct: ['96.46', '73.68']
Recall: ['91.80', '87.20']
IoU: ['88.82', '66.49']
F1 Score: ['94.08', '79.87']
mean IoU: 77.65
FCN test result:
global correct: 90.09
average row correct: ['95.14', '72.54']
Recall: ['92.32', '81.15']
IoU: ['88.16', '62.08']
F1 Score: ['93.71', '76.61']
mean IoU: 75.12


In [ ]:
from torchvision.utils import save_image
from dataset import reconstruct_patched

!mkdir {RESULT_DIR}/test_image
model.eval()
with torch.no_grad():
  for idx, (test_image, _, grid, _) in enumerate(test_dataset):
    inputs = test_image.to(device)
    outputs = model(inputs)['out']
    preds = 1-outputs.argmax(dim=1).cpu().detach()
    filename = f"{os.path.splitext(test_dataset.filenames[idx])[0]}.png"
    pred_path = os.path.join(RESULT_DIR, "test_image", filename)
    save_image(1-reconstruct_patched(preds, grid).float(), pred_path)